## Problem Statement
TAKE THE IRIS DATA SET, OBTAINED FROM THE UNIVERSITY OF CALIFORNIA-IRVINE MACHINE LEARNING REPOSITORY (LINK PROVIDED IN THE REFERENCE SECTION), AS A DATA SET TO BE DISCRETIZED. PERFORM DATA DISCRETIZATION FOR EACH OF FOUR NUMERIC ATTRIBUTE USING CHIMERGE METHOD. (LET THE STOPPING CRITERIA BE: MAX-INTERVAL 6). YOU NEED TO WRITE A SMALL PYTHON PROGRAM RO DO THIS TO AVOID CLUMSY NUMERICAL COMPUTATIONS. SUBMIT YOUR SIMPLE ANALYSIS AND YOUR TEST RESULTS: SPLIT-POINTS, FINAL INTERVALS AND THE WELL DOCUMENTED SOURCE PROGRAM IN PYTHON JUPYTER NOTEBOOK. 

Objective :
1. SPLIT POINTS
2. FINAL INTERVALS
3. CHI-MERGE SOURCE

## Solution

### Read Iris Data from the web

> IRIS Data set [http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data] can be read with the help of pandas read_csv into a dataframe

> Set the Column/Header Names as
    1. 'Sepal_Length' 
    2. 'Sepal_Width' 
    3. 'Petal_Length'
    4. 'Petal_Width' 
    5. 'Class'

In [1]:
#Imports for the implementation
import pandas as pd
import numpy as np
import math
from collections import Counter
from prettytable import PrettyTable

In [2]:
# Load the IRIS Data into the DataFrame
iris_frame = pd.read_csv(
    'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    header=None)
iris_frame.columns = [
    'Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Class'
]

### Perform the Chi-Merge algorithm

#### Algorithm Psuedo Code: 
1. Sort the Data set with the help of feature set

2. Pick the unique values as intervals. Refer Pandas qcut[https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html]

3. Iterate till the intervals are greater than the input interval

   + Iterate over the intervals and pick adjacent intervals
   + Calculate the chi distance between the picked intervals   
   + Identify the Minimum chi value from the calculations
   + Perform the Merger of the adjacents

\begin{equation}
\chi^2=\Sigma\frac{(O-E)^2}{E} \\
\text{Where O is the actual value and E is the expected value.}
\end{equation}

### Source - ChiMerge


In [3]:
class ChiMerge:

    __author__ = 'Vijay Karamsetty'
    '''
        Takes the dataset and nominal col for the discretization and
    '''
    def __init__(self, data_set, nominal_col, verbose=False):
        self.data = data_set
        self.nominal_col = nominal_col
        self.verbose = verbose
        if nominal_col != None:
            self.nominal_set = set(self.data[nominal_col])
        print(self)

    '''
        Step 1 : Method will be used to sort the dataset with the help of given column/feature
    '''
    def get_sorted_data_set(self, data_set, feature):
        if feature != None:
            return data_set.sort_values(by=[feature])
        #Return the Dataset as is
        return data_set

    '''
        Step 2: Calculated chisquare for the observations of two intervals
         a. Count the observations on the each data set into observation rows for the classes
         b. Summarize the observations into row sums and column sums
         c. Calculate the expected as row_sum(i)* column_sum(j) / total observations
         d. Calculate the chi as sum of all (observed-expected)**2/expected
    '''
    def chi_square(self, observation_bin1, observation_bin2, nominal_col,
                   feature, nominal_set):
        expected_dictionary = {nominal: 0 for nominal in nominal_set}
        observation_row1 = np.array([
            cnt_observations for cls, cnt_observations in {
                **expected_dictionary,
                **Counter(observation_bin1[nominal_col])
            }.items()
        ])
        observation_row2 = np.array([
            cnt_observations for cls, cnt_observations in {
                **expected_dictionary,
                **Counter(observation_bin2[nominal_col])
            }.items()
        ])
        all_observations = len(observation_row1) + len(observation_row2)
        interval_observation_sum = observation_row1 + observation_row2
        e_00 = interval_observation_sum * sum(
            observation_row1) / all_observations
        e_11 = interval_observation_sum * sum(
            observation_row2) / all_observations

        chi_2 = np.zeros(interval_observation_sum.shape)

        if np.all(e_00):
            chi_2 += (observation_row1 - e_00)**2 / e_00

        if np.all(e_11):
            chi_2 += (observation_row2 - e_11)**2 / e_11

        return chi_2

    '''
        Implementing using group by of dataframes
    '''
    def chi_2(self, bin1, bin2, nominal_col, feature, nominal_set):
        grouping_arr = []
        expected_dictionary = {nominal: 0 for nominal in nominal_set}

        observation_row1 = np.array([
            (bin1[bin1[nominal_col] == i][feature].count())
            for i in expected_dictionary
        ])
        observation_row2 = np.array([
            (bin2[bin2[nominal_col] == i][feature].count())
            for i in expected_dictionary
        ])

        all_observations = len(observation_row1) + len(observation_row2)
        interval_observation_sum = observation_row1 + observation_row2
        e_00 = interval_observation_sum * sum(
            observation_row1) / all_observations
        e_11 = interval_observation_sum * sum(
            observation_row2) / all_observations

        chi_2 = np.zeros(interval_observation_sum.shape)

        if np.all(e_00):
            chi_2 += (observation_row1 - e_00)**2 / e_00

        if np.all(e_11):
            chi_2 += (observation_row2 - e_11)**2 / e_11

        return chi_2

    '''
        Performs the merge of i with adjacent i.e. i+1 interval and results into a new one with
        Interval(i.left, i+1.right)
    '''
    def merge_indices(self, intervals, index):
        # New Interval List
        new_intervals = []
        try:
            skip_next = False
            for i in range(len(intervals)):
                #left of i, right of i+1 is the new interval
                if (i == index) or (i - 1 == index):
                    if not skip_next:
                        new_intervals.append(
                            pd.Interval(intervals[i].left,
                                        intervals[i + 1].right))
                        skip_next = True
                else:
                    new_intervals.append(intervals[i])
        except ValueError:
            print('Invalid index manipulation')
        return new_intervals

    def get_distinct_bins(self, data_set, feature):
        #Identify the Unique data for the feature
        unique_set_len = len(set(data_set[feature]))

        # Split into bins with the help of unique set len that is provided
        bins, init_split_points = pd.qcut(data_set[feature],
                                          q=unique_set_len,
                                          retbins=True,
                                          duplicates='drop')
        if self.verbose:
            print(['Intervals'], bins)

        return bins.cat.categories

    '''
        Step 3 : chi-merge for the feature
    '''
    def chi_merge(self, feature, max_intervals):
        # Step 1 : Get the Sorted Dataset
        data_set = self.get_sorted_data_set(self.data, feature)
        #Get the intervals for the discretization
        intervals = self.get_distinct_bins(data_set, feature)
        #perfrom looping till the intervals are reducted to the max_intervals
        while len(intervals) > max_intervals:
            interim_chi_values = []
            for i in range(len(intervals) - 1):
                bin1 = data_set[data_set[feature].between(
                    intervals[i].left, intervals[i].right)]
                bin2 = data_set[data_set[feature].between(
                    intervals[i + 1].left, intervals[i + 1].right)]
                chi_2 = self.chi_2(bin1, bin2, self.nominal_col, feature,
                                        self.nominal_set)
                #chi_2 = self.chi_2(bin1, bin2, self.nominal_col, feature,
                #                       self.nominal_set)
                
                interim_chi_values.append(sum(chi_2))
            if len(interim_chi_values) > 0:
                intervals = self.merge_indices(
                    intervals,
                    interim_chi_values.index(min(interim_chi_values)))

        return intervals

    def __str__(self):
        return "ChiMerge v[{}]".format(1.0)

In [4]:
# Nominal Column used
nominal_col = 'Class'
cobj = ChiMerge(data_set=iris_frame, nominal_col=nominal_col, verbose=False)

for feature in ['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width']:
    split_points = cobj.chi_merge(feature, max_intervals=6)
    print("Discretization Intervals for Feature {}".format(feature))
    x = PrettyTable()
    x.field_names = ['Lower', 'Upper']
    for each in split_points:
        x.add_row([each.left, each.right])
    print(x)

ChiMerge v[1.0]
Discretization Intervals for Feature Sepal_Length
+--------------------+-------+
|       Lower        | Upper |
+--------------------+-------+
| 4.2989999999999995 |  4.8  |
|        4.8         | 4.929 |
|       4.929        |  5.5  |
|        5.5         |  5.7  |
|        5.7         |  5.9  |
|        5.9         |  7.9  |
+--------------------+-------+
Discretization Intervals for Feature Sepal_Width
+-------+-------+
| Lower | Upper |
+-------+-------+
| 1.999 |  2.8  |
|  2.8  |  3.0  |
|  3.0  |  3.1  |
|  3.1  |  3.2  |
|  3.2  |  3.5  |
|  3.5  |  4.4  |
+-------+-------+
Discretization Intervals for Feature Petal_Length
+-------+-------+
| Lower | Upper |
+-------+-------+
| 0.999 |  4.4  |
|  4.4  |  5.8  |
|  5.8  |  5.96 |
|  5.96 |  6.1  |
|  6.1  | 6.507 |
| 6.507 |  6.9  |
+-------+-------+
Discretization Intervals for Feature Petal_Width
+-------+-------+
| Lower | Upper |
+-------+-------+
| 0.099 |  1.3  |
|  1.3  |  2.0  |
|  2.0  |  2.1  |
|  2.1  